# Project Steps
 ## Main Steps:

* Camera calibration and distortion correction.
* Color/gradient threshold.
* Perspective transform.
* Detect lane lines.

## Extra Step:
* Determine the lane curvature.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy.polynomial.polynomial as poly
%matplotlib inline

In [ ]:
import os
test_dir = "test_images"
test_files = os.listdir(test_dir)
test_files

# Camera Calibration  and Distortion Correction


*   Utils


In [ ]:
def cameraCalibrate():
    # Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
    import glob

    #read in and make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')

    #Arrays to store objects points and image points from all the images
    objpoints = [] #3d points
    imgpoints = [] #2d points

    #prepare points
    objp  = np.zeros((9*6,3),np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)


    for fname in images:
        #read in each image
        img = mpimg.imread(fname)
        #convert image to gray scale  
        gray  = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        #find corners  
        ret,corners = cv2.findChessboardCorners(gray,(9,6),None)
        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)
            
    shape =(img.shape[1],img.shape[0])
    ret, mtx, dist, rvect, tvect = cv2.calibrateCamera(objpoints,imgpoints,shape,None,None)
    
    #undistored the image
    # undistorted_image = cv2.undistort(image, mtx, dist, None, mtx)
    return mtx,dist,mtx

* Method

In [ ]:
camCalibration = cameraCalibrate()

def undistort_image(image, verbose=0):
    output = cv2.undistort(image, camCalibration[0], camCalibration[1], None, camCalibration[2])
    if (verbose > 0):
        f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('Original image', fontsize=20)
        ax1.imshow(image)
        ax2.set_title('calibrated image', fontsize=20)
        ax2.imshow(output)
    return output
    

* Apply (Calibrate)

In [ ]:
filename = 'test5.jpg'
sample_image = mpimg.imread(test_dir + "/" + filename)
# sample_image = mpimg.imread('./camera_cal/calibration1.jpg')

f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Original image', fontsize=20)
ax1.imshow(sample_image)
ax2.set_title('calibrated image', fontsize=20)
calibrated_image=undistort_image(sample_image)
ax2.imshow(undistort_image(sample_image))

# Gradient / Color Threshold + Sobel

* Utils

In [ ]:
def visualize(ch1, ch2, ch3,
            ch1_name, ch2_name, ch3_name,
            plot):
    plot[0].set_title(ch1_name + '_channel', fontsize=20)
    plot[0].imshow(ch1,cmap='gray')
    plot[1].set_title(ch2_name + '_channel', fontsize=20)
    plot[1].imshow(ch2,cmap='gray')
    plot[2].set_title(ch3_name + '_channel', fontsize=20)
    plot[2].imshow(ch3,cmap='gray')

def binarize(array, l_thresh, u_thresh, yes=1, no=0):
    binary = (array >= l_thresh) * array
    binary = (array <= u_thresh) * binary
    binary = (binary > 0)*yes
    return binary

# Color threshold for the white line
def white(img, channel_num = 0, white_thresh = (220,255)):
    channel = img[:,:,channel_num]    
    l_thresh, u_thresh = white_thresh[:]
    binary = binarize(channel, l_thresh, u_thresh, yes=255, no=0)
    return binary

# Color threshold for the yellow line
def yellow(img, channel_num = 2, yellow_thresh = (220,255)):
    channel = img[:,:,channel_num]
    l_thresh, u_thresh = yellow_thresh[:]
    # binary = cv2.inRange(img, np.array([220, 220, 220]), np.array([255, 255, 255]))
    binary = binarize(channel, l_thresh, u_thresh, yes=255, no=0)
    return binary

* Possible Models

-- HLS Model

-- HSV Model

-- LAB Model

-- RGB Model

In [ ]:
image_in_action = np.copy(calibrated_image)
f, plot_axs = plt.subplots(4, 3, figsize=(20,20))

hls_image = cv2.cvtColor(image_in_action,cv2.COLOR_RGB2HLS)
h_channel = hls_image[:,:,0]
l_channel = hls_image[:,:,1]
s_channel = hls_image[:,:,2]
visualize(h_channel, l_channel, s_channel, "h", "l", "s", plot_axs[0])

hsv_image = cv2.cvtColor(image_in_action,cv2.COLOR_RGB2HSV)
h_channel = hsv_image[:,:,0]
s_channel = hsv_image[:,:,1]
v_channel = hsv_image[:,:,2]
visualize(h_channel, s_channel, v_channel, "h", "s", "v", plot_axs[1])

lab_image = cv2.cvtColor(image_in_action,cv2.COLOR_RGB2LAB)
l_channel = lab_image[:,:,0]
a_channel = lab_image[:,:,1]
b_channel = lab_image[:,:,2]
visualize(l_channel, a_channel, b_channel, "l", "a", "b", plot_axs[2])

rgb_image = image_in_action
r_channel = rgb_image[:,:,0]
g_channel = rgb_image[:,:,1]
b_channel = rgb_image[:,:,2]
visualize(r_channel, g_channel, b_channel, "r", "g", "b", plot_axs[3])

* Sobel

In [ ]:
# X or Y sobel gradient
def abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # TODO: Convert to grayscale using cv2.COLOR_RGB2GRAY as the conversion
    if len(image.shape) == 2:
        gray_img = image
    else:
        gray_img = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    # TODO: Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value    
    if orient == 'x':
        abs_sobel = np.abs(cv2.Sobel(gray_img,cv2.CV_64F,1,0,ksize=sobel_kernel))
    else:
        abs_sobel = np.abs(cv2.Sobel(gray_img,cv2.CV_64F,0,1,ksize=sobel_kernel))
   
    # TODO: Rescale back to 8 bit integer
    abs_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # TODO: Create a binary image of ones where threshold is met, zeros otherwise
#     abs_sobel_output = np.zeros_like(abs_sobel)
#     abs_sobel_output = abs_sobel[abs_sobel > thresh[0] or abs_sobel < thresh[1]].astype(int)
    abs_sobel_output = binarize(abs_sobel, thresh[0], thresh[1], yes=255)
    # Return the binary image
    return abs_sobel_output

* Method (USING SOBEL OVER COLOR MODELS, USING COLOR THRESHOLDING)

In [ ]:
# def get_binary_thresholded_img(calibrated_image, verbose=0, gaussian_kernel = (0,0),
#                                 yellow_thresh=(170,200), white_thresh=(220,255), sobel_thresh=(40,55), sobel_kernel=5):
#     image_in_action = calibrated_image.copy()
#     image_in_action = cv2.GaussianBlur(image_in_action, (0,0), cv2.BORDER_DEFAULT)

#     lab_image = cv2.cvtColor(image_in_action,cv2.COLOR_RGB2LAB)
#     hsv_image = cv2.cvtColor(image_in_action,cv2.COLOR_RGB2HSV)

#     # binary_white = white(hsv_image,  white_thresh=(220,255),channel_num=2)
#     # binary_yellow = yellow(lab_image, yellow_thresh=(170,200),channel_num=2)
#     # x_sobel = abs_sobel_thresh(image_in_action, orient='x', sobel_kernel=5, thresh=(45, 55))
#     # y_sobel = abs_sobel_thresh(image_in_action, orient='y', sobel_kernel=5, thresh=(45, 55))
#     # combine both the white, yellow and x_sobel binaries in combined_binary

#     binary_white_x = abs_sobel_thresh(hsv_image[:,:,2], orient='x', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
#     binary_yellow_x = abs_sobel_thresh(lab_image[:,:,2], orient='x', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
#     binary_white_y = abs_sobel_thresh(hsv_image[:,:,2], orient='y', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
#     binary_yellow_y = abs_sobel_thresh(lab_image[:,:,2], orient='y', sobel_kernel=sobel_kernel, thresh=sobel_thresh)

#     binary_combined = np.zeros_like(image_in_action)
#     # binary_combined[(binary_white >= 1) | (binary_yellow >= 1) | (x_sobel >= 1)|(y_sobel >= 1)] = 255
#     binary_combined[(binary_white_x >= 1) | (binary_yellow_x >= 1) | (binary_white_y >= 1) | (binary_yellow_y >= 1)] = 255
#     if verbose > 0:
#         # visualize
#         f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20,10))
#         ax1.set_title('binary white x', fontsize=20)
#         ax1.imshow(binary_white_x, cmap='gray')
#         ax2.set_title('binary white y', fontsize=20)
#         ax2.imshow(binary_white_y, cmap='gray')
#         ax3.set_title('binary yellow y', fontsize=20)
#         ax3.imshow(binary_yellow_y, cmap='gray')
#         ax4.set_title('binary yellow x', fontsize=20)
#         ax4.imshow(binary_yellow_x, cmap='gray')

#         f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
#         ax1.set_title('image_of_application', fontsize=20)
#         ax1.imshow(image_in_action, cmap='gray')
#         ax2.set_title('combined', fontsize=20)
#         ax2.imshow(binary_combined, cmap='gray')
#     return binary_combined

In [ ]:
def get_binary_thresholded_img(calibrated_image, verbose=0, gaussian_kernel = (0,0),
                                yellow_thresh=(170,200), white_thresh=(220,255), sobel_thresh=(40,55), sobel_kernel=5):
    img = calibrated_image.copy()
    image_in_action = cv2.GaussianBlur(img, gaussian_kernel, cv2.BORDER_DEFAULT)

    lab_image = cv2.cvtColor(img,cv2.COLOR_RGB2LAB)
    hsv_image = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)

    binary_white = white(hsv_image,  white_thresh=white_thresh,channel_num=2)
    binary_yellow = yellow(lab_image, yellow_thresh=yellow_thresh,channel_num=2)
    x_sobel = abs_sobel_thresh(img, orient='x', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    # y_sobel = abs_sobel_thresh(img, orient='y', sobel_kernel=sobel_kernel, thresh=sobel_thresh)

    # combine both the white, yellow and x_sobel binaries in combined_binary
    binary_combined = np.zeros_like(image_in_action)
    # binary_combined[(binary_white >= 1) | (binary_yellow >= 1) | (x_sobel >= 1) | (y_sobel >= 1)] = 255
    binary_combined[(binary_white >= 1) | (binary_yellow >= 1) | (x_sobel >= 1)] = 255

    if verbose > 0:
        # visualize
        f, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(20,10))
        ax1.set_title('binary white', fontsize=20)
        ax1.imshow(binary_white, cmap='gray')
        ax2.set_title('binary yellow', fontsize=20)
        ax2.imshow(binary_yellow, cmap='gray')
        ax3.set_title('x_sobel', fontsize=20)
        ax3.imshow(x_sobel, cmap='gray')
        # ax4.set_title('y_sobel', fontsize=20)
        # ax4.imshow(y_sobel, cmap='gray')
        f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('blurred image', fontsize=20)
        ax1.imshow(image_in_action, cmap='gray')
        ax2.set_title('combined', fontsize=20)
        ax2.imshow(binary_combined, cmap='gray')

    return binary_combined

* Apply

In [ ]:

image_in_action = calibrated_image.copy()
image_in_action = cv2.GaussianBlur(image_in_action, (0,0), cv2.BORDER_DEFAULT)

lab_image = cv2.cvtColor(image_in_action,cv2.COLOR_RGB2LAB)
hsv_image = cv2.cvtColor(image_in_action,cv2.COLOR_RGB2HSV)

# binary_white = white(hsv_image,  white_thresh=(220,255),channel_num=2)
# binary_yellow = yellow(lab_image, yellow_thresh=(170,200),channel_num=2)
# x_sobel = abs_sobel_thresh(image_in_action, orient='x', sobel_kernel=5, thresh=(45, 55))
# y_sobel = abs_sobel_thresh(image_in_action, orient='y', sobel_kernel=5, thresh=(45, 55))
# combine both the white, yellow and x_sobel binaries in combined_binary

binary_white_x = abs_sobel_thresh(hsv_image[:,:,2], orient='x', sobel_kernel=5, thresh=(45, 55))
binary_yellow_x = abs_sobel_thresh(lab_image[:,:,2], orient='x', sobel_kernel=5, thresh=(45, 55))
binary_white_y = abs_sobel_thresh(hsv_image[:,:,2], orient='y', sobel_kernel=5, thresh=(45, 55))
binary_yellow_y = abs_sobel_thresh(lab_image[:,:,2], orient='y', sobel_kernel=5, thresh=(45, 55))

binary_combined = np.zeros_like(image_in_action)
# binary_combined[(binary_white >= 1) | (binary_yellow >= 1) | (x_sobel >= 1)|(y_sobel >= 1)] = 255
binary_combined[(binary_white_x >= 1) | (binary_yellow_x >= 1) | (binary_white_y >= 1) | (binary_yellow_y >= 1)] = 255

# visualize
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20,10))
ax1.set_title('binary white x', fontsize=20)
ax1.imshow(binary_white_x, cmap='gray')
ax2.set_title('binary white y', fontsize=20)
ax2.imshow(binary_white_y, cmap='gray')
ax3.set_title('binary yellow y', fontsize=20)
ax3.imshow(binary_yellow_y, cmap='gray')
ax4.set_title('binary yellow x', fontsize=20)
ax4.imshow(binary_yellow_x, cmap='gray')

f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('image_of_application', fontsize=20)
ax1.imshow(image_in_action, cmap='gray')
ax2.set_title('combined', fontsize=20)
ax2.imshow(binary_combined, cmap='gray')

# Perspective Transform

* Utils (Method)

In [ ]:
def perspective_transform(undistorted, inverse=False, verbose=0):   

    img = np.copy(undistorted)
    w = img.shape[1]
    h = img.shape[0]
    img_size = (w, h)
    src = np.float32([[0,h - 2],[3.6*w/8,3.7*h/6],
                  [4.4*w/8,3.7*h/6],[w,h - 2]])
    # make sure that the points follow the right arrangement whether it's clockwise or counter-clockwise
    # source and destination points must have the same arrangement whether it's clockwise or counter-clockwise
    # The points in src array are (x,y).
    dst = np.float32([[2*w/10,h -2],[2*w/10,2*h/10],
                    [8*w/10,2*h/10],[8*w/10,h - 2]])

    if(inverse):
        M = cv2.getPerspectiveTransform(dst, src)
    else:
        M = cv2.getPerspectiveTransform(src, dst)

    transformed_img = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    # gray_scale = False

    # if len(img.shape) == 2:
    #     print(img.shape)
    #     gray_scale = True
    #     img = cv2.cvtColor(undistorted, cv2.COLOR_GRAY2RGB)
    # M = cv2.getPerspectiveTransform(src, dst)
    # transformed_img = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)


    # if gray_scale:
    #     transformed_img = cv2.cvtColor(transformed_img, cv2.COLOR_RGB2GRAY)

    # visualize your mask region
    if(verbose > 0):
        f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
        ax1.set_title('source area', fontsize=20)
        ax1.imshow(undistorted)
        ordered_y = [src[0][1],src[1][1],src[2][1],src[3][1],src[0][1]]
        ordered_x = [src[0][0],src[1][0],src[2][0],src[3][0],src[0][0]]
        ax1.plot(ordered_x,ordered_y , color='red', alpha=0.7,
            linewidth=3, solid_capstyle='round', zorder=2)

        # visualize your mask region
        f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
        ax1.set_title('destination area', fontsize=20)
        ax1.imshow(undistorted)
        ordered_y = [dst[0][1],dst[1][1],dst[2][1],dst[3][1],dst[0][1]]
        ordered_x = [dst[0][0],dst[1][0],dst[2][0],dst[3][0],dst[0][0]]
        ax1.plot(ordered_x,ordered_y , color='red', alpha=0.7,
            linewidth=3, solid_capstyle='round', zorder=2)

        # visualize your results
        f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('Original image', fontsize=20)
        ax1.imshow(undistorted)
        ax2.set_title('Transformed image', fontsize=20)
        ax2.imshow(transformed_img)

    return transformed_img

* Apply (Transform)

In [ ]:
# Undistord the image then apply perspective transformation
transformed_image = perspective_transform(calibrated_image, verbose=1)


# Detect Lane Lines

After finishing the previous steps You now have a thresholded warped image and you're ready to map out the lane lines! There are many ways you could go about this, but here's one example of how you might do it:
### Peaks in a Histogram and Sliding Windows
* After applying calibration, thresholding, and a perspective transform to a road image, you should have a binary image where the lane lines stand out clearly. However, you still need to decide explicitly which pixels are part of the lines and which belong to the left line and which belong to the right line.
* we can use the two highest peaks from our histogram as a starting point for determining where the lane lines are, and then use sliding windows moving upward in the image (further along the road) to determine where the lane lines go.
#### steps:
  1. split the histogram into two sides, one for each lane line.
  2. Set up sliding windows and window hyperparameters:
     * set a few hyperparameters related to our sliding windows, and set them up to iterate across the binary activations in the image. These hyperparameters are:
        1. **W_Number**; number of sliding windows.
        2. **Margin**; the width of each window.
        3. **Minimum_pixels**; used as a threshold to recenter the next sliding window.
        4. **Window_Height**; computed from number of pixels and image height.
  3. Loop through each window in W_Number.
  4. Find the boundaries of our current window. This is based on a combination of the current window's starting point      , as well as the margin you set in the hyperparameters.
  5. Use cv2.rectangle to draw these window boundaries onto visualization image.
  6. Now that we know the boundaries of our window, find out which activated(non zero) pixels actually fall into the window.
  7. Append these non zero pixels to two different arrays one for the right line and the other for the left line.
  8. If the number of pixels you found in Step **6** are greater than your hyperparameter Minimum_pixels, re-center our window based on the mean position of these pixels.
  9. Now that we have found all our pixels belonging to each line through the sliding window method, it's time to fit a polynomial to the line.

* Sliding Window (Method)

In [ ]:
def detectlanes(w_number, margin, minimum_pixels, binary_img, left_peak=-1, right_peak=-1, verbose=0):

    out_img = binary_img.copy()
    # get the image's width and height.
    w = out_img.shape[1]
    h = out_img.shape[0]

    if(left_peak < 0 or right_peak < 0):
        left_peak = w//4
        right_peak = w * 3 //4

    # 2. Set up sliding windows and window hyperparameters:
    window_height = int(h // w_number)

    half_window_height = int(window_height // 2)
    half_window_width = int(margin // 1.5)
    current_height = int(h - half_window_height)

    left_window_center = int(left_peak)
    right_window_center = int(right_peak)
    left_fit = np.int32(np.empty((0,2)))
    right_fit = np.int32(np.empty((0,2)))
    
    
    # 3. Loop through each window in W_Number.
    for i in range(0, w_number):
        if(i > 0):
            half_window_width = int(margin // 2)
        left_window_boundaries = [( left_window_center - half_window_width, current_height + half_window_height) , (left_window_center + half_window_width, current_height - half_window_height)]
        right_window_boundaries = [(right_window_center - half_window_width, current_height + half_window_height) , (right_window_center + half_window_width, current_height - half_window_height)]
        # print(left_window_boundaries[0])
        # print(left_window_boundaries[1])
        
        # print(left_window_boundaries[0])
        # left_window = out_img[left_window_boundaries[1][1]: left_window_boundaries[0][1],left_window_boundaries[0][0]:left_window_boundaries[1][0],:]
        # right_window = out_img[right_window_boundaries[1][1]: right_window_boundaries[0][1],right_window_boundaries[0][0]:right_window_boundaries[1][0],:]
        left_window_indices, left_horizontal_indcies = window_helper(left_window_boundaries[1][1],left_window_boundaries[0][1],left_window_boundaries[0][0],left_window_boundaries[1][0], out_img)


        right_window_indices, right_horizontal_indcies = window_helper(right_window_boundaries[1][1],right_window_boundaries[0][1],right_window_boundaries[0][0],right_window_boundaries[1][0], out_img)

        # print(current_height)
        # print(right_window_center)
        if(len(left_horizontal_indcies) > minimum_pixels):
            left_window_center = int(np.mean(left_horizontal_indcies))
            left_window_boundaries = [( left_window_center - half_window_width, current_height + half_window_height) , (left_window_center + half_window_width, current_height - half_window_height)]

        if(len(right_horizontal_indcies) > minimum_pixels):
            right_window_center = int(np.mean(right_horizontal_indcies))
            right_window_boundaries = [(right_window_center - half_window_width, current_height + half_window_height) , (right_window_center + half_window_width, current_height - half_window_height)]



        


        if(current_height > 0):
            left_fit = np.append(left_fit, [(left_window_center,current_height)], axis=0)
            right_fit = np.append(right_fit,[(right_window_center,current_height)] , axis =0)

        # if(i == 0):
        #     left_fit = left_window_indices
        #     right_fit = right_window_indices
        # else:
        #     left_fit = np.append(left_fit, left_window_indices, axis=0)
        #     right_fit = np.append(right_fit, right_window_indices , axis =0)

        for index in left_fit:
            # print(index)
            cv2.circle(out_img, tuple(index), 3, (255, 0, 0) , -1) 

        for index in right_fit:
            cv2.circle(out_img, tuple(index), 3, (255, 0, 0) , -1)    
        # print(left_window_boundaries[0])
        cv2.rectangle(out_img, left_window_boundaries[0], left_window_boundaries[1], (0, 255, 0), 2)
        cv2.rectangle(out_img, right_window_boundaries[0], right_window_boundaries[1], (0, 255, 0), 2)
        cv2.circle(out_img, (left_window_center,current_height), 2, (255, 0, 0) , -1) 
        cv2.circle(out_img, (right_window_center,current_height), 2, (255, 0, 0) , -1) 
 

     


        current_height -= (window_height + 4)

    if(verbose > 0):
        f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
        ax1.set_title('Sliding Windows', fontsize=20)

        ax1.imshow(out_img)
    # print(left_fit.shape)
    return left_fit, right_fit

def window_helper(min_height, max_height, min_width, max_width, img):
    test = img[min_height:max_height,min_width:max_width,:]
    # print(img_copy.shape)
    # print(test.shape)
    x,y,z = np.where(test == (255,255,255))
    x += min_height
    y += min_width
    # print(x)
    # print(y)
    # mean = 0
    # if(len(x) > min_pixels):
    #     mean = (int(np.mean(x)), int(np.mean(y)))
    arr = np.array((y,x))

    # print(arr[0])
    # print(arr[1])
    # final = np.reshape(np.ravel(arr),(-1,2),order='F')
    return arr, y

* Histogram Peaks (Method)

In [ ]:
# binary_white = white(calibrated_image, channel_num=0, white_thresh = (245,255))
# binary_yellow = yellow(lab_image, channel_num=2, yellow_thresh = (170,200))

# # combine both the white and yellow binaries in combined_binary
# binary_combined = np.zeros_like(image_in_action)
# binary_combined[(binary_white >= 1) | (binary_yellow >= 1)] = 255

def get_histogram_peaks(binary_transformed_image, percentage=1, verbose=0):
    # print(binary_transformed_image.shape)
    image_histo = np.zeros_like(binary_transformed_image)
    image_histo[binary_transformed_image >= 1] = 1 
    # print(image_histo[:,:,1])
    image_histo = image_histo[:,:,1]
    row_start_index = int(image_histo.shape[0]*(1-percentage))
    image_histo = image_histo[row_start_index:]
    image_histo = image_histo.reshape(image_histo.shape[1], image_histo.shape[0])
    image_histo = image_histo.sum(axis=1)
    # print(image_histo)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Transformed Image', fontsize=20)
    ax1.imshow(binary_transformed_image)
    ax2.set_title('Histogram', fontsize=20)
    # x_values = np.array([i for i in range(0, binary_transformed_image.shape[1])])
    # print(binary_transformed_image.shape)
    x_new = np.linspace(0, binary_transformed_image.shape[1], num=binary_transformed_image.shape[1])

    ax2.plot(x_new, image_histo)

    image_center = int(image_histo.shape[0]/2)
    left_peak = np.argmax(image_histo[:image_center])
    right_peak = np.argmax(image_histo[image_center:]) + image_center
    print("Left peak %s, Right peak %s" % (left_peak, right_peak))
    return left_peak, right_peak
# print(left_fit.shape)



* Apply (Using histogram)

In [ ]:
transformed_image = perspective_transform(calibrated_image)
binary_transformed_image = get_binary_thresholded_img(transformed_image, white_thresh=(200,255), yellow_thresh=(140,200),verbose=1)
left_peak,right_peak = get_histogram_peaks(binary_transformed_image, percentage=0.2, verbose=1)
left_fit, right_fit = detectlanes(20,200,10,binary_transformed_image, left_peak=left_peak, right_peak=right_peak, verbose=1)

* Apply (Not using histogram)

In [ ]:
left_fit, right_fit = detectlanes(20,200,10,binary_transformed_image, verbose=1)

## Drawing the lane

* Draw Lane (Method)

In [ ]:
def draw_lane(img,bird_eye,left_fit,right_fit):
    tmp_image     = np.copy(img)
    if right_fit is None or left_fit is None:
        return img
    
    zero          = np.zeros_like(bird_eye).astype(np.uint8)
    layered_image = np.dstack((zero,zero,zero))
    
    ploty      = np.linspace(0, bird_eye.shape[0]-1, bird_eye.shape[0] )

    left_fitx  = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2] 
    
    #formatting the points
    left   = np.array([np.transpose(np.vstack([right_fitx,ploty]))])
    right  = np.array([np.flipud(np.transpose(np.vstack([left_fitx,ploty])))])
    points = np.hstack((left,right))
    # print(np.int_([points]))
    
    #form lane
    cv2.fillPoly(layered_image,np.int32([points]),(0,255,0))
    cv2.polylines(layered_image,np.int32([right]),isClosed = False,color=(255,0,0),thickness = 20)
    cv2.polylines(layered_image,np.int32([left]),isClosed = False,color=(255,0,0),thickness = 20)
    
    # The inverse perspective transfom note
    # use the inverse perspective option mentioned in the note above to transform back the layered_image
    inversed = perspective_transform(layered_image, inverse=True)
    
    output   = cv2.addWeighted(tmp_image,1,inversed,0.5,0)
    return output

In [ ]:

# left_fit_x, left_fit_y, (right_fit_x, right_fit_y = detectlanes(20,200,10,left_peak,right_peak, transformed_image)
def draw_lane_updated(img,bird_eye,left_fit,right_fit, verbose=0):
    tmp_image = np.copy(img)
    zero          = np.zeros_like(bird_eye).astype(np.uint8)
    layered_image = bird_eye
    x_new = np.linspace(0, bird_eye.shape[0]-1, bird_eye.shape[0] )
    # x_new = np.linspace(left_fit_x[0], left_fit_x[-1], num=len(left_fit_x)*10)

    # coefs = poly.polyfit(left_fit_y,left_fit_x ,2)
    # ffit = poly.polyval(x_new, coefs)
    # plt.plot(x_new, ffit)
    # # print(left_fit_x[0])
    # plt.plot(right_fit_x,right_fit_y)

    left_coefs = poly.polyfit(left_fit[:,1],left_fit[:,0] ,2)
    right_coefs = poly.polyfit(right_fit[:,1],right_fit[:,0] ,2)

    left_fit_values = poly.polyval(x_new, left_coefs)
    right_fit_values = poly.polyval(x_new, right_coefs)

    
    # print(left_fit[:,0])
    # print(left_fit[:,1])
    # print(right_fit[:,0])
    # print(right_fit[:,1])

    # print(right_fit_values)
    # print(right_fit_values)

    # print(left_fit[:,0])
    # print(left_fit[:,1])
        # visualize 
    if(verbose > 0):
        f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('Left Fit data on transformed image', fontsize=20)
        ax1.plot(left_fit_values, x_new)
        ax1.plot(left_fit[:,0],left_fit[:,1])
        ax1.imshow(bird_eye)
        # ax1.imshow(image_in_action)

        ax2.set_title('Right Transformed image', fontsize=20)
        ax2.plot(right_fit_values, x_new)
        ax2.plot(right_fit[:,0],right_fit[:,1])
        ax2.imshow(bird_eye)

    zero = np.zeros_like(bird_eye).astype(np.uint8)


    arr = np.array((left_fit_values,x_new))
    left = np.reshape(np.ravel(arr),(-1,2),order='F')
    arr = np.array((right_fit_values,x_new))
    right = np.reshape(np.ravel(arr),(-1,2),order='F')
    points = np.append(left,np.flip(right,axis=0),axis=0)

    # print(np.int32(points))
    cv2.polylines(zero,np.int32([left]),isClosed = False,color=(255,0,0),thickness = 20)
    cv2.polylines(zero,np.int32([right]),isClosed = False,color=(255,0,0),thickness = 20)
    # cv2.polylines(bird_eye_view,np.int32([points]),isClosed = False,color=(255,0,0),thickness = 20)
    cv2.fillPoly(zero,np.int32([points]),(0,255,0))
    inversed = perspective_transform(zero, inverse=True)    
    # plt.imshow(tmp_image)
    # plt.imshow(img)



        # ax2.imshow(image_in_action)
    output = cv2.addWeighted(tmp_image,1,inversed,0.5,0)
    return output




In [ ]:
def get_final_image(input_image, verbose=0):
    sub_verbose = verbose - 1
    calibrated_image = undistort_image(input_image, sub_verbose - 1)
    bird_eye_view = perspective_transform(calibrated_image, verbose=sub_verbose - 1)
    binary_img = get_binary_thresholded_img(bird_eye_view,verbose=sub_verbose,yellow_thresh=(150,200),white_thresh=(220,255))
    # cv2.imshow("Test.jpg",calibrated_image)
    # cv2.waitKey(0) 
    
    left_fit, right_fit = detectlanes(7,200,4,binary_img, verbose=sub_verbose)
    output_img = draw_lane_updated(calibrated_image,binary_img,left_fit, right_fit, verbose=sub_verbose)
    np_output_img = np.asarray(output_img)

    f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Original image', fontsize=20)
    ax1.imshow(calibrated_image)
    ax2.set_title('Detected image', fontsize=20)
    ax2.imshow(np_output_img)
    return output_img
# cv2.imshow("Test.jpg",output_img)
# cv2.waitKey(0) 

## Determine The Lane Curvature
You're getting very close to a final result! You have a thresholded image, where you've estimated which pixels belong to the left and right lane lines, and you've fit a polynomial to those pixel positions. Next we'll compute the radius of curvature of the fit.

## Curvature in Pixels
In the last step we computed the lane line pixels using their x and y pixel positions to fit a second order polynomial curve: $$f(y) = Ay^2+By+C $$
in this step you will compute the radius of curvature at the closest point to the vehicle.

**Radius of Curvature Equation:**
$$R\_Curve = \frac{[1+(\frac{dx}{dy})^2]^{3/2}}{|\frac{d^2x}{dy^2}|}$$

$$f'(y) = \frac{dx}{dy} = 2Ay+B$$

$$f''(y) = \frac{d^2x}{dy^2} =A$$

## From Pixels to Real World
* Great! You've now calculated the radius of curvature for our lane lines. But now we need to stop and think... We've calculated the radius of curvature based on pixel values, so the radius we are reporting is in pixel space, which is not the same as real world space. So we actually need to repeat this calculation after converting our x and y values to real world space.

* This involves measuring how long and wide the section of lane is that we're projecting in our warped image. We could do this in detail by measuring out the physical lane in the field of view of the camera, but for this project, you can assume that if you're projecting a section of lane similar to the images above, the lane is about 30 meters long and 3.7 meters wide.

In [ ]:
def calc_r_curve(active_pixels):
    active_pixels = active_pixels.reshape(active_pixels.shape[1], active_pixels.shape[0])

    x = active_pixels[0]
    y = active_pixels[1]
    p = np.polyfit(x, y, 2)
    a, b, c = p

    dx_dy_coff = np.polyder(p)
    dx_dy_2_coff = np.polyder(p, m=2)

    print("JUST TEST")
    print(dx_dy_coff)
    print(dx_dy_2_coff)

    dx_dy = 2*a*y + b
    dx_dy_2 = a

    numerator = (1 + dx_dy**2)**(3/2)
    denominator = np.abs(dx_dy_2)
    r_curve = numerator/denominator
    return r_curve

# Final Image 

In [ ]:
# Get the final image 
get_final_image(sample_image, verbose=2)

# Load all images

In [ ]:
def load_images(path):
    import glob

    #read in and make a list of calibration images
    images = glob.glob(path)
    output_images = []
    for fname in images:
        #read in each image
        img = mpimg.imread(fname)   
        output_images.append(img)

    return output_images

* Apply on multiple images

In [ ]:
all_images = load_images('./test_images/*.jpg')

In [ ]:
for img in all_images:
    get_final_image(img, verbose=2)

In [ ]:
# left_r_curve = calc_r_curve(left_active_pixels)
# right_r_curve = calc_r_curve(right_active_pixels)

# left_fit_xy = left_fit.reshape(left_fit.shape[1], left_fit.shape[0])
# right_fit_xy = right_fit.reshape(right_fit.shape[1], right_fit.shape[0])
# print(right_fit_xy[1][0])
# right_fit_xy = np.flip(right_fit_xy, axis=1)


# print(right_fit_xy[1][right_fit_xy.shape[1]-1])
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
# ax1.set_title('left lane original', fontsize=20)
# ax1.plot(left_fit_xy[0],left_fit_xy[1])
# ax2.set_title('right lane original', fontsize=20)
# ax2.plot(right_fit_xy[0],right_fit_xy[1])
# print(left_fit_xy.shape)

# # # NOT SURE YET
# # left_fit = left_r_curve
# # right_fit = right_r_curve
# left_xs = [left_p1[0], left_p2[0]]
# left_ys = [left_p1[1], left_p2[1]]
# right_xs = [right_p1[0], right_p2[0]]
# right_ys = [right_p1[1], right_p2[1]]
# print("points")
# print(left_xs)
# print(left_ys)
# left_fit_p = np.polyfit(left_fit[0], left_fit[1], 2)
# right_fit_p = np.polyfit(right_fit[0], right_fit[1], 2)
# print("polynomials")
# print(left_fit_p)
# print(right_fit_p)
# bird_eye_view = perspective_transform(calibrated_image)
# defined_lane_image = None

# defined_lane_image = draw_lane(calibrated_image, bird_eye_view, left_fit_p, right_fit_p)
# # print(defined_lane_image)
# # print(defined_lane_image.shape)


# layered_image = np.zeros_like(bird_eye_view)
# left_p1 = np.array([left_fit_xy[0][0], left_fit_xy[1][0]])
# left_p2 = np.array([left_fit_xy[0][left_fit_xy.shape[1]-1], left_fit_xy[1][left_fit_xy.shape[1]-1]])
# right_p1 = np.array([right_fit_xy[0][0], right_fit_xy[1][0]])
# right_p2 = np.array([right_fit_xy[0][right_fit_xy.shape[1]-1], right_fit_xy[1][right_fit_xy.shape[1]-1]])
# left = [left_p1, left_p2]
# right = [right_p1, right_p2]
# points = np.array([left_p1, left_p2, right_p1, right_p2])
# print(points)
# cv2.fillPoly(layered_image, np.int32([points]), (0,255,0))
# cv2.polylines(layered_image,np.int32([right]),isClosed = False,color=(255,0,0),thickness = 20)
# cv2.polylines(layered_image,np.int32([left]),isClosed = False,color=(255,0,0),thickness = 20)



# # The inverse perspective transfom note
# # use the inverse perspective option mentioned in the note above to transform back the layered_image
# inversed = perspective_transform(layered_image, inverse=True)

# tmp_image     = np.copy(calibrated_image)
# output   = cv2.addWeighted(tmp_image,1,inversed,0.5,0)


# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
# ax1.set_title('manual lines-in-prespective', fontsize=20)
# ax1.imshow(layered_image)
# ax2.set_title('auto LANE Image transform', fontsize=20)
# ax2.imshow(perspective_transform(defined_lane_image))

# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
# ax1.set_title('MANUAL DETECTED LANE Image', fontsize=20)
# ax1.imshow(output)
# ax2.set_title('POLY DETECTED LANE Image', fontsize=20)
# ax2.imshow(defined_lane_image)
